In [2]:
from tensorflow.python.keras.datasets import imdb
from tensorflow.python.keras.preprocessing import sequence

In [3]:
vocab_size = 5000
sentence_size = 200
embedding_size = 50
modir = '.'

pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2

In [4]:
(x_train_variable, y_train), (x_test_variable, y_test) = imdb.load_data(
    num_words=vocab_size, start_char=start_id, oov_char=oov_id,
    index_from=index_offset)

In [86]:
len(x_train_variable[0])

218

In [14]:
x_train = sequence.pad_sequences(x_train_variable,
                                maxlen=sentence_size,
                                truncating='post',
                                padding='post',
                                value=pad_id)
x_test = sequence.pad_sequences(x_test_variable,
                               maxlen=sentence_size,
                               truncating='post',
                               padding='post',
                               value=pad_id)

In [16]:
x_train.shape,x_test.shape

((25000, 200), (25000, 200))

In [17]:
word_index = imdb.get_word_index()

In [20]:
word_inverted_index = {v + index_offset: k for k,v in word_index.items()}

In [25]:
word_inverted_index[pad_id] = '<PAD>'
word_inverted_index[start_id] = '<START>'
word_inverted_index[oov_id] = '<OOV>'

In [27]:
def index_to_text(indexes):
    return ' '.join([word_inverted_index[i] for i in indexes])
print(index_to_text(x_train_variable[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <OOV> is an amazing actor and now the same being director <OOV> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <OOV> and would recommend it to everyone to watch and the fly <OOV> was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <OOV> to the two little <OOV> that played the <OOV> of norman and paul they were just brilliant children are often left out of the <OOV> list i think because the stars that play them all grown up are such a big <OOV> for the whole film but these children are amazing and should be <OOV> for what they

In [29]:
import numpy as np
x_len_train = np.array([min(len(x), sentence_size) for x in x_train_variable])
x_len_test = np.array([min(len(x), sentence_size) for x in x_test_variable])

In [32]:
def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

In [41]:
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_train, x_len_train, y_train))
    dataset = dataset.shuffle(buffer_size=len(x_train_variable))
    dataset = dataset.batch(32)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    return dataset.make_one_shot_iterator().get_next()

In [34]:
def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_test, x_len_test, y_test))
    dataset = dataset.batch(32)
    dataset = dataset.map(parser)
    return dataset.make_one_shot_iterator().get_next()

In [36]:
import tensorflow as tf
column = tf.feature_column.categorical_column_with_identity('x', vocab_size)

In [51]:
indicator = tf.feature_column.indicator_column(column)
#tensor = tf.feature_column.input_layer(x_train, [indicator])
next_batch_features, next_batch_labels = train_input_fn()
with tf.Session() as sess:
    fa = sess.run(next_batch_features)
    la = sess.run(next_batch_labels)
    tensor = tf.feature_column.input_layer(fa, [indicator])
    ret = sess.run(tensor)

In [55]:
ret[0]

array([86.,  1.,  5., ...,  0.,  0.,  0.], dtype=float32)

In [58]:
x_train[0].shape

(200,)

In [63]:
import os
classifier = tf.estimator.LinearClassifier(feature_columns=[column], model_dir=os.path.join('.', 'bow_sparse'))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './bow_sparse', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x120a66a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [66]:
from tensorboard import summary as summary_lib
all_classifiers = {}
def train_and_evaluate(classifier):
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn, steps=25000)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    predictions = np.array([p['logistic'][0] for p in classifier.predict(input_fn=eval_input_fn)])
    
    tf.reset_default_graph()
    pr = summary_lib.pr_curve('prcision_recall', predictions=predictions, labels=y_test.astype(bool), num_thresholds=21)
    with tf.Session() as sess:
        writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
        writer.add_summary(sess.run(pr), global_step=0)
        writer.close()

In [67]:
tf.logging.set_verbosity(tf.logging.INFO)
train_and_evaluate(classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./bow_sparse/model.ckpt.
INFO:tensorflow:loss = 22.18071, step = 1
INFO:tensorflow:global_step/sec: 149.237
INFO:tensorflow:loss = 25.610685, step = 101 (0.671 sec)
INFO:tensorflow:global_step/sec: 415.365
INFO:tensorflow:loss = 8.042709, step = 201 (0.240 sec)
INFO:tensorflow:global_step/sec: 427.594
INFO:tensorflow:loss = 9.701427, step = 301 (0.234 sec)
INFO:tensorflow:global_step/sec: 426.774
INFO:tensorflow:loss = 16.21009, step = 401 (0.235 sec)
INFO:tensorflow:global_step/sec: 425.755
INFO:tensorflow:loss = 15.258985, step = 501 (0.235 sec)
INFO:tensorflow:global_step/sec: 423.481
INFO:tensorflow:loss = 5.0844936, step = 601 (0.236 sec)
INFO:tensorflow:global_step/sec: 407.519
INFO:tensorflow:loss = 1

INFO:tensorflow:global_step/sec: 419.088
INFO:tensorflow:loss = 8.515694, step = 8101 (0.238 sec)
INFO:tensorflow:global_step/sec: 397.459
INFO:tensorflow:loss = 2.7466795, step = 8201 (0.252 sec)
INFO:tensorflow:global_step/sec: 441.305
INFO:tensorflow:loss = 9.611645, step = 8301 (0.227 sec)
INFO:tensorflow:global_step/sec: 416.879
INFO:tensorflow:loss = 7.1844378, step = 8401 (0.240 sec)
INFO:tensorflow:global_step/sec: 403.153
INFO:tensorflow:loss = 4.532151, step = 8501 (0.247 sec)
INFO:tensorflow:global_step/sec: 418.294
INFO:tensorflow:loss = 7.921661, step = 8601 (0.239 sec)
INFO:tensorflow:global_step/sec: 388.402
INFO:tensorflow:loss = 8.14988, step = 8701 (0.257 sec)
INFO:tensorflow:global_step/sec: 334.631
INFO:tensorflow:loss = 8.96896, step = 8801 (0.301 sec)
INFO:tensorflow:global_step/sec: 407.135
INFO:tensorflow:loss = 8.2070465, step = 8901 (0.244 sec)
INFO:tensorflow:global_step/sec: 426.076
INFO:tensorflow:loss = 7.791809, step = 9001 (0.234 sec)
INFO:tensorflow:glo

INFO:tensorflow:global_step/sec: 401.826
INFO:tensorflow:loss = 4.3993073, step = 16401 (0.248 sec)
INFO:tensorflow:global_step/sec: 372.84
INFO:tensorflow:loss = 3.3620293, step = 16501 (0.269 sec)
INFO:tensorflow:global_step/sec: 465.458
INFO:tensorflow:loss = 2.2286756, step = 16601 (0.215 sec)
INFO:tensorflow:global_step/sec: 448.141
INFO:tensorflow:loss = 5.639369, step = 16701 (0.223 sec)
INFO:tensorflow:global_step/sec: 428.956
INFO:tensorflow:loss = 2.7227945, step = 16801 (0.233 sec)
INFO:tensorflow:global_step/sec: 438.654
INFO:tensorflow:loss = 7.358714, step = 16901 (0.227 sec)
INFO:tensorflow:global_step/sec: 444.04
INFO:tensorflow:loss = 2.9160895, step = 17001 (0.226 sec)
INFO:tensorflow:global_step/sec: 434.571
INFO:tensorflow:loss = 4.933096, step = 17101 (0.229 sec)
INFO:tensorflow:global_step/sec: 429.871
INFO:tensorflow:loss = 6.5939865, step = 17201 (0.233 sec)
INFO:tensorflow:global_step/sec: 376.022
INFO:tensorflow:loss = 5.601712, step = 17301 (0.266 sec)
INFO:t

INFO:tensorflow:loss = 2.8444347, step = 24601 (0.228 sec)
INFO:tensorflow:global_step/sec: 440.752
INFO:tensorflow:loss = 2.4472308, step = 24701 (0.227 sec)
INFO:tensorflow:global_step/sec: 442.67
INFO:tensorflow:loss = 4.739454, step = 24801 (0.226 sec)
INFO:tensorflow:global_step/sec: 416.247
INFO:tensorflow:loss = 2.561214, step = 24901 (0.240 sec)
INFO:tensorflow:Saving checkpoints for 25000 into ./bow_sparse/model.ckpt.
INFO:tensorflow:Loss for final step: 4.1885614.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-06-06:57:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./bow_sparse/model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-06-06:57:27
INFO:tensorflow:Saving dict for global step 25000: accuracy = 0.8228, accuracy_baseline = 0.5, auc = 0.89903724, auc_precision_recall = 0.

In [69]:
weights = classifier.get_variable_value('linear/linear_model/x/weights').flatten()

In [71]:
sorted_indexes = np.argsort(weights)
extremes = np.concatenate((sorted_indexes[-8:], sorted_indexes[:8]))
extreme_weights = sorted([(weights[i], word_inverted_index[i]) for i in extremes])

In [72]:
extreme_weights

[(-3.0273607, 'disappointment'),
 (-2.9678218, 'forgettable'),
 (-2.9615827, 'stinker'),
 (-2.632478, 'horrid'),
 (-2.5348005, 'unwatchable'),
 (-2.487474, 'slightest'),
 (-2.4847324, 'obnoxious'),
 (-2.4045312, 'boredom'),
 (2.2584417, 'flawless'),
 (2.323624, 'edie'),
 (2.3818674, 'soccer'),
 (2.4856758, 'paulie'),
 (2.5508673, 'showdown'),
 (2.7764611, 'voight'),
 (2.8585005, 'timeless'),
 (3.285095, 'refreshing')]

In [73]:
word_embedding_column = tf.feature_column.embedding_column(column, dimension=embedding_size)
classifier = tf.estimator.DNNClassifier(
    hidden_units=[100],
    feature_columns=[word_embedding_column],
    model_dir=os.path.join('.', 'bow_embeddings'))
train_and_evaluate(classifier)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './bow_embeddings', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x177ce7eb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./bow_embeddings/model.ckpt.
INFO:tensorflow:loss = 22.14887

INFO:tensorflow:loss = 4.7108297, step = 7401 (0.315 sec)
INFO:tensorflow:global_step/sec: 288.395
INFO:tensorflow:loss = 8.791243, step = 7501 (0.347 sec)
INFO:tensorflow:global_step/sec: 304.19
INFO:tensorflow:loss = 7.656868, step = 7601 (0.328 sec)
INFO:tensorflow:global_step/sec: 314.423
INFO:tensorflow:loss = 8.974883, step = 7701 (0.318 sec)
INFO:tensorflow:global_step/sec: 251.668
INFO:tensorflow:loss = 6.3540564, step = 7801 (0.397 sec)
INFO:tensorflow:global_step/sec: 284.504
INFO:tensorflow:loss = 3.8994071, step = 7901 (0.352 sec)
INFO:tensorflow:global_step/sec: 316.136
INFO:tensorflow:loss = 6.0163016, step = 8001 (0.316 sec)
INFO:tensorflow:global_step/sec: 326.672
INFO:tensorflow:loss = 2.5151513, step = 8101 (0.306 sec)
INFO:tensorflow:global_step/sec: 313.959
INFO:tensorflow:loss = 6.98451, step = 8201 (0.319 sec)
INFO:tensorflow:global_step/sec: 325.125
INFO:tensorflow:loss = 4.2807856, step = 8301 (0.308 sec)
INFO:tensorflow:global_step/sec: 321.036
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 205.534
INFO:tensorflow:loss = 5.4121656, step = 15701 (0.486 sec)
INFO:tensorflow:global_step/sec: 311.395
INFO:tensorflow:loss = 6.2299147, step = 15801 (0.321 sec)
INFO:tensorflow:global_step/sec: 249.121
INFO:tensorflow:loss = 3.5735126, step = 15901 (0.401 sec)
INFO:tensorflow:global_step/sec: 250.537
INFO:tensorflow:loss = 4.5852833, step = 16001 (0.401 sec)
INFO:tensorflow:global_step/sec: 228.193
INFO:tensorflow:loss = 2.0522008, step = 16101 (0.436 sec)
INFO:tensorflow:global_step/sec: 318.974
INFO:tensorflow:loss = 5.3052435, step = 16201 (0.313 sec)
INFO:tensorflow:global_step/sec: 246.817
INFO:tensorflow:loss = 6.5197062, step = 16301 (0.406 sec)
INFO:tensorflow:global_step/sec: 217.743
INFO:tensorflow:loss = 1.8677287, step = 16401 (0.458 sec)
INFO:tensorflow:global_step/sec: 241.08
INFO:tensorflow:loss = 3.1392188, step = 16501 (0.415 sec)
INFO:tensorflow:global_step/sec: 236.648
INFO:tensorflow:loss = 7.34166, step = 16601 (0.425 sec)
INF

INFO:tensorflow:loss = 1.054802, step = 23901 (0.489 sec)
INFO:tensorflow:global_step/sec: 238.88
INFO:tensorflow:loss = 1.8584158, step = 24001 (0.417 sec)
INFO:tensorflow:global_step/sec: 293.068
INFO:tensorflow:loss = 2.5254917, step = 24101 (0.341 sec)
INFO:tensorflow:global_step/sec: 244.839
INFO:tensorflow:loss = 1.5059445, step = 24201 (0.410 sec)
INFO:tensorflow:global_step/sec: 197.107
INFO:tensorflow:loss = 2.3587995, step = 24301 (0.506 sec)
INFO:tensorflow:global_step/sec: 269.595
INFO:tensorflow:loss = 1.7209024, step = 24401 (0.370 sec)
INFO:tensorflow:global_step/sec: 306.878
INFO:tensorflow:loss = 1.8658566, step = 24501 (0.327 sec)
INFO:tensorflow:global_step/sec: 243.45
INFO:tensorflow:loss = 1.3683448, step = 24601 (0.409 sec)
INFO:tensorflow:global_step/sec: 271.753
INFO:tensorflow:loss = 2.283667, step = 24701 (0.368 sec)
INFO:tensorflow:global_step/sec: 301.477
INFO:tensorflow:loss = 1.6013665, step = 24801 (0.332 sec)
INFO:tensorflow:global_step/sec: 256.415
INFO

In [80]:
head = tf.contrib.estimator.binary_classification_head()
def cnn_model_fn(features, labels, mode, params):
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size, initializer=params['embedding_initializer'])
    training = mode == tf.estimator.ModeKeys.TRAIN
    dropout_emb = tf.layers.dropout(inputs=input_layer,
                                   rate=0.2,
                                   training=training)
    conv = tf.layers.conv1d(
        inputs=dropout_emb,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    dropout_hidden = tf.layers.dropout(inputs=hidden,
                                        rate=0.2,
                                        training=training)
    
    logits = tf.layers.dense(inputs=dropout_hidden, units=1)
    
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits,
        train_op_fn=_train_op_fn)

In [81]:
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                       model_dir=os.path.join('.', 'cnn'),
                                       params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './cnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1210196a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [82]:
train_and_evaluate(cnn_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./cnn/model.ckpt.
INFO:tensorflow:loss = 0.80238384, step = 1
INFO:tensorflow:global_step/sec: 36.0262
INFO:tensorflow:loss = 0.7254535, step = 101 (2.776 sec)
INFO:tensorflow:global_step/sec: 41.1484
INFO:tensorflow:loss = 0.6798403, step = 201 (2.431 sec)
INFO:tensorflow:global_step/sec: 44.7914
INFO:tensorflow:loss = 0.6412723, step = 301 (2.232 sec)
INFO:tensorflow:global_step/sec: 45.1721
INFO:tensorflow:loss = 0.65594214, step = 401 (2.214 sec)
INFO:tensorflow:global_step/sec: 43.1932
INFO:tensorflow:loss = 0.5096842, step = 501 (2.316 sec)
INFO:tensorflow:global_step/sec: 47.3366
INFO:tensorflow:loss = 0.45693558, step = 601 (2.114 sec)
INFO:tensorflow:global_step/sec: 49.3537
INFO:tensorflow:loss = 0

INFO:tensorflow:global_step/sec: 49.9301
INFO:tensorflow:loss = 0.12565072, step = 8001 (2.002 sec)
INFO:tensorflow:global_step/sec: 50.1425
INFO:tensorflow:loss = 0.18635392, step = 8101 (1.995 sec)
INFO:tensorflow:global_step/sec: 50.6686
INFO:tensorflow:loss = 0.17552707, step = 8201 (1.973 sec)
INFO:tensorflow:global_step/sec: 44.6749
INFO:tensorflow:loss = 0.08805633, step = 8301 (2.238 sec)
INFO:tensorflow:global_step/sec: 46.5923
INFO:tensorflow:loss = 0.09695908, step = 8401 (2.147 sec)
INFO:tensorflow:global_step/sec: 44.6564
INFO:tensorflow:loss = 0.14272107, step = 8501 (2.238 sec)
INFO:tensorflow:global_step/sec: 50.4428
INFO:tensorflow:loss = 0.32874364, step = 8601 (1.982 sec)
INFO:tensorflow:global_step/sec: 48.2985
INFO:tensorflow:loss = 0.3069429, step = 8701 (2.071 sec)
INFO:tensorflow:global_step/sec: 46.8587
INFO:tensorflow:loss = 0.21938601, step = 8801 (2.134 sec)
INFO:tensorflow:global_step/sec: 49.7285
INFO:tensorflow:loss = 0.28003708, step = 8901 (2.011 sec)
I

INFO:tensorflow:loss = 0.087984025, step = 16101 (2.151 sec)
INFO:tensorflow:global_step/sec: 48.8429
INFO:tensorflow:loss = 0.041647438, step = 16201 (2.047 sec)
INFO:tensorflow:global_step/sec: 48.7116
INFO:tensorflow:loss = 0.14985123, step = 16301 (2.053 sec)
INFO:tensorflow:global_step/sec: 49.9408
INFO:tensorflow:loss = 0.06041684, step = 16401 (2.002 sec)
INFO:tensorflow:global_step/sec: 48.1868
INFO:tensorflow:loss = 0.06404889, step = 16501 (2.075 sec)
INFO:tensorflow:global_step/sec: 48.6929
INFO:tensorflow:loss = 0.03870105, step = 16601 (2.054 sec)
INFO:tensorflow:global_step/sec: 50.0227
INFO:tensorflow:loss = 0.07283878, step = 16701 (1.999 sec)
INFO:tensorflow:global_step/sec: 49.4573
INFO:tensorflow:loss = 0.099653125, step = 16801 (2.022 sec)
INFO:tensorflow:global_step/sec: 48.2101
INFO:tensorflow:loss = 0.119814746, step = 16901 (2.074 sec)
INFO:tensorflow:global_step/sec: 47.8678
INFO:tensorflow:loss = 0.020976765, step = 17001 (2.089 sec)
INFO:tensorflow:global_ste

INFO:tensorflow:loss = 0.06504112, step = 24201 (2.233 sec)
INFO:tensorflow:global_step/sec: 46.6911
INFO:tensorflow:loss = 0.03224159, step = 24301 (2.142 sec)
INFO:tensorflow:global_step/sec: 49.6278
INFO:tensorflow:loss = 0.0015350173, step = 24401 (2.015 sec)
INFO:tensorflow:global_step/sec: 50.4615
INFO:tensorflow:loss = 0.011022596, step = 24501 (1.983 sec)
INFO:tensorflow:global_step/sec: 49.1834
INFO:tensorflow:loss = 0.12431011, step = 24601 (2.032 sec)
INFO:tensorflow:global_step/sec: 50.2376
INFO:tensorflow:loss = 0.07947795, step = 24701 (1.990 sec)
INFO:tensorflow:global_step/sec: 49.0608
INFO:tensorflow:loss = 0.0067690336, step = 24801 (2.038 sec)
INFO:tensorflow:global_step/sec: 48.838
INFO:tensorflow:loss = 0.015451647, step = 24901 (2.048 sec)
INFO:tensorflow:Saving checkpoints for 25000 into ./cnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0061596786.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation 